RL on a taxi 

In [1]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,086 kB of archives.
After this operation, 5,413 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig4.0 amd64 4.0.1-5build1 [1,081 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig all 4.0.1-5build1 [5,528 B]
Fetched 1,086 kB in 0s (7,628 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.1-5build1_amd64.deb ...
Unpacking swig4.0 (4.0.1-5build1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.1-5build1_a

In [2]:
!pip install setuptools==65.5.0
!pip install stable-baselines3[extra]
!pip install box2d-py
!pip install huggingface_sb3
!pip install pyglet
!pip install gym[box2d]
!sudo apt-get update
!pip install gym
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.1
    Uninstalling setuptools-67.6.1:
      Successfully uninstalled setuptools-67.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.1 requires setuptools>65.5.1, but you have setuptools 65.5.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [3]:
import gym
from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login
from collections import deque
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import os
from stable_baselines3.common.vec_env import VecFrameStack


In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [5]:
env = gym.make("Taxi-v3")

In [6]:
!pip uninstall -y pyglet
!pip install pyglet==1.5.0

# Create environment
env = gym.make("Taxi-v3")

observation = env.reset()

for _ in range(10):
    #take random action
    action = env.action_space.sample()
    print("Action taken:", action)
    # Render the environment
    env.render()

    observation, reward, done, info = env.step(action)

    #if the game is done
    if done:
        #Reset the environment
        observation = env.reset()
        print("Environment reset")



Found existing installation: pyglet 2.0.5
Uninstalling pyglet-2.0.5:
  Successfully uninstalled pyglet-2.0.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
Action taken: 3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action taken: 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Action taken: 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Action taken: 4
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Action taken: 4
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
Action taken: 3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
Action taken: 1
+---------+
|R: | : :G|
| : | : : |
| : : : :

In [7]:
env = make_vec_env("Taxi-v3", n_envs = 10)

In [8]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize



In [9]:
import os

# Create a directory for logs
if not os.path.exists("./logs"):
    os.mkdir("./logs")

# Set the log path
log_path = "./logs/PPO_1"


In [14]:
# Create environment
env = gym.make("Taxi-v3")


model = PPO(
    policy="MlpPolicy",
    env=env,
    learning_rate=0.00005,  # lower learning rate for stability
    n_steps=2048,  # increase n_steps for more efficient training
    batch_size=64,  # increase batch size for more efficient training
    n_epochs=1000,
    gamma=0.99,  # adjust gamma to balance immediate vs future rewards
    gae_lambda=0.95,  # adjust lambda to balance bias vs variance in advantage estimation
    clip_range=0.25,  # adjust clip range for policy gradient optimization
    clip_range_vf=None,  # disable clipping for value function optimization
    ent_coef=0.0,  # adjust entropy coefficient to balance exploration vs exploitation
    vf_coef=0.7,  # adjust value function coefficient to balance policy gradient and value function optimization
    max_grad_norm=0.6,  # setting gradient clipping
    tensorboard_log= log_path,
    verbose=1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
print(model)

In [12]:
import time

In [16]:
# Train the agent for 10000 timesteps
start = time.time()
model.learn(total_timesteps=10000)
end = time.time()

# Print the total training time
print(f"Total training time: {end - start:.2f} seconds")

Logging to ./logs/PPO_1/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 199      |
|    ep_rew_mean     | -781     |
| time/              |          |
|    fps             | 822      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -762        |
| time/                   |             |
|    fps                  | 25          |
|    iterations           | 2           |
|    time_elapsed         | 160         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.039979722 |
|    clip_fraction        | 0.351       |
|    clip_range           | 0.25        |
|    entropy_loss         | -1.76       |
|    explained_variance   | -0.000184   |


In [17]:
# save the model 
model_name = "taxi_model"
model.save(model_name)

In [19]:

# Create a new environment for evaluation
eval_env = gym.make('Taxi-v3')

# Evaluate the model 
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True , render= True)

# Print the results

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :